In [1]:
import math, sys
from konlpy.tag import Okt

베이지안 필터 클래스 

긍/부정도 구분 가능할 듯.

In [4]:
class BayesianFilter:
    """베이지안 필터"""
    def __init__(self):
        self.words = set() # 출현한 단어 기록
        self.word_dict = {} # 카테고리마다의 출현 기록
        self.category_dict = {} # 카테고리 출현 횟수 기록 
        
        
    # 형태소 분석
    def txt_split(self, text): 
        results = []
        okt = Okt()
        
        #단어 기본형 사용
        malist = okt.pos(text, norm=True, stem=True)
        for word in malist:
            # 어미/조사/구두점 등은 대상에서 제외
            if not word[1] in ["Josa", "Eomi", "Punctuation"]:
                results.append(word[0])
        
        return results
    
    
    # 단어와 카테고리 출현 횟수 기록
    def cnt_word(self, word, category):
        # 단어를 카테고리에 추가 
        if not category in self.word_dict:
            self.word_dict[category] = {}
            
        if not word in self.word_dict[category]:
            self.word_dict[category][word] = 0
            
        self.word_dict[category][word] += 1
        self.words.add(word)
        
    
    def cnt_category(self, category):
        #카테고리 기록
        if not category in self.category_dict:
            self.category_dict[category] = 0
        
        self.category_dict[category] += 1
        
    
    def fit(self, text, category):
        """텍스트 학습"""
        word_list = self.txt_split(text)
        
        for word in word_list:
            self.cnt_word(word, category)
        
        self.cnt_category(category)
    
    
    # 단어 리스트 점수 기록
    def score(self, words, category):
        score = math.log(self.category_prob(category)) # 확률을 곱할 때 값이 너무 작으면 다운 플로 발생하므로 log함수 사용 
        
        for word in words:
            score += math.log(self.word_prob(word, category))
        
        return score
     
        
    # 예측 
    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize
        words = self.txt_split(text)
        score_list = []
        
        for category in self.category_dict.keys():
            score = self.score(words, category)
            score_list.append((category, score))
            
            if score > max_score:
                max_score = score
                best_category = category
        
        return best_category, score_list
    
    
    # 카테고리 내부의 단어 출현 횟수 구하기 
    def get_word_count(self, word, category):
        
        if word in self.word_dict[category]:
            return self.word_dict[category][word]
    
        else:
            return 0
    
    # 카테고리 계산
    def category_prob(self, category):
        sum_categories = sum(self.category_dict.values())
        category_v = self.category_dict[category]
        
        return category_v / sum_categories
    
    
    # 카테고리 내부의 단어 출현 비율 계산
    def word_prob(self, word, category):
        n = self.get_word_count(word, category) + 1 # 학습 사전에 없는 단어가 나오면 확률이 0이 되므로 이를 방지하기 위해 1을 더한다. 
        d = sum(self.word_dict[category].values()) + len(self.words)
        
        return n / d

Bayes Test

In [5]:
bf = BayesianFilter()

# 텍스트 학습
bf.fit("파격 세일 - 오늘까지만 30% 할인", "광고")
bf.fit("쿠폰 선물 & 무료 배송", "광고")
bf.fit("현대 신세계 백화점 세일", "광고")
bf.fit("봄과 함께 찾아온 따뜻한 신제품 소식", "광고")
bf.fit("인기 제품 기간 한정 세일", "광고")
bf.fit("오늘 일정 확인", "중요")
bf.fit("프로젝트 진행 상황 보고", "중요")
bf.fit("계약 잘 부탁드립니다.", "중요")
bf.fit("회의 일정이 등록되었습니다.", "중요")
bf.fit("오늘 일정이 없습니다.", "중요")


# 예측
pre, scorelist = bf.predict("재고 정리 할인, 무료 배송")
print("결과 :", pre)
print("점수 :", scorelist)

결과 : 광고
점수 : [('광고', -19.329379270837773), ('중요', -20.544606748320554)]
